In [8]:
import zipfile

kmz_file_path = 'C:/Users/elfak/Downloads/LISTE VILLAGES (1).kmz'
extracted_folder_path = 'C:/Users/elfak/Downloads/'

with zipfile.ZipFile(kmz_file_path, 'r') as kmz:
    kmz.extractall(extracted_folder_path)


In [3]:
import xml.etree.ElementTree as ET
import pandas as pd

# Define the KML namespace
namespace = "{http://www.opengis.net/kml/2.2}"

# Parse the KML data
kml_file_path = 'C:/Users/elfak/Downloads/doc.kml'  # Replace with your actual path
tree = ET.parse(kml_file_path)
root = tree.getroot()

# Initialize lists to store the extracted data
names = []
coordinates = []

# Extract data from Placemark elements using the namespace
for placemark in root.findall(f'.//{namespace}Placemark'):
    name = placemark.find(f'.//{namespace}name').text.strip()
    coordinates_text = placemark.find(f'.//{namespace}coordinates').text.strip()
    # Split the coordinates string into latitude, longitude, and altitude
    coordinates_list = coordinates_text.split(',')[:2]
    longitude, latitude = map(float, coordinates_list)
    
    names.append(name)
    coordinates.append((latitude, longitude))

# Create a DataFrame
data = {'name': names, 'latitude': [coord[0] for coord in coordinates], 'longitude': [coord[1] for coord in coordinates]}
df = pd.DataFrame(data)

# Save the data to an Excel file
excel_file_path = 'locations.xlsx'  # Adjust the file path as needed
df.to_excel(excel_file_path, index=False)

print(f'Data saved to {excel_file_path}')


Data saved to locations.xlsx


In [185]:
df_location = pd.read_excel('locations.xlsx')
df_location

,name,latitude,longitude
0,Douar Tazloumt,30.808782,-8.606285
1,Talat N'Yaaqoub,30.993130,-8.235383
2,Ijoukak,30.997324,-8.162536
3,دوار امندار‎,30.847017,-8.579649
4,دوار تجكَالت‎,31.060757,-8.855264
...,...,...,...
328,مجموعة مدارس تكنزا - وحدة أنومس‎,30.875672,-8.787370
329,مسجد انادن,30.983637,-8.152778
330,Agdim,31.306409,-6.854532
331,Tazelga,31.237979,-7.879984


In [12]:
%autoreload

import pandas as pd
from sources.utils import update_excel, merge_by_distance_coords


UsageError: Line magic function `%autoreload` not found.


In [231]:
df = pd.read_excel('earthquake_relief.xlsx')
df_location = pd.read_excel('locations.xlsx')

# df_merged = merge_by_distance_coords(df,df_location,5)
# df_merged # .to_excel('temp_csv.xlsx')

# Processing data

In [238]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points
from geopy.distance import geodesic
def merge_by_distance_coords_(df1,df_location,distance_threshold_km):
# Convert the DataFrame to Shapely Point geometries
    df1['geometry'] = [Point(lon, lat) for lon, lat in zip(df1['longitude'], df1['latitude'])]
    df_location['geometry'] = [Point(lon, lat) for lon, lat in zip(df_location['longitude'], df_location['latitude'])]

    merged_data = []

    for index1, row1 in df1.iterrows():
        for index2, row2 in df_location.iterrows():
            # Calculate the distance between the two points
            distance = geodesic((row1['latitude'], row1['longitude']), (row2['latitude'], row2['longitude'])).kilometers
            if distance <= distance_threshold_km:
                merged_data.append({**row1, **row2, 'Distance (km)': distance})
            # else : 
            #     zero_list = {'Vehicle': None, 'Aid': None, 'Location': row1['Location'], 'Count': 0}
            #     merged_data.append({**zero_list, **row2, 'Distance (km)': distance})

    # Create a new DataFrame from the merged data
    merged_df = pd.DataFrame(merged_data)
    # rest_df = df_location[~df_location['geometry'].isin(merged_df['geometry'])]
    # rest_df['Aid'] = None
    # rest_df['Vehicle'] = None
    # rest_df['Count'] = 0
    # rest_df['Location'] = rest_df['longitude'].astype(str) + ', ' + rest_df['latitude'].astype(str)
    # rest_df['Distance (km)'] = None
    # rest_df = rest_df[['Vehicle', 'Aid', 'Location', 'Count', 'latitude', 'longitude', 'geometry', 'name', 'Distance (km)']]
    # result = pd.concat([merged_df,rest_df])
    return merged_df

In [244]:
df_merged = merge_by_distance_coords_(df,df_location,3).drop_duplicates()[['name', 'Vehicle', 'Count', 'Aid', 'location', 'latitude', 'longitude',
       'geometry', 'Distance (km)']]
df_merged  # .to_excel('temp_csv.xlsx')

,name,Vehicle,Count,Aid,location,latitude,longitude,geometry,Distance (km)
0,Adassil,شاحنة صغيرة,1,الرعاية الصحية,"31.0896185, -8.4818423",31.112307,-8.496595,POINT (-8.496595 31.112307),2.882479


In [245]:
from shapely import wkt 

df_merged['geometry_str'] = df_merged['geometry'].astype(str)
total_aids_by_type = df_merged.groupby(['Aid', 'Vehicle','geometry_str'])['Count'].sum().reset_index()
# total_aids_by_type['geometry'] = total_aids_by_type['geometry_str'].apply(wkt.loads)

def join_lists(lst):
    return ', '.join(lst)

test_df = total_aids_by_type.groupby(['geometry_str','Aid','Vehicle'])[['Count']].agg(int).reset_index()
test_df['stats'] = test_df['Aid'] + " : " + test_df['Vehicle'] + " (" + test_df['Count'].astype(str) + ")"

test_df = test_df.groupby(['geometry_str'])[['Aid','Vehicle','Count','stats']].agg(list).reset_index()
test_df['Aid'] = test_df['Aid'].apply(join_lists)
test_df['Vehicle'] = test_df['Vehicle'].apply(join_lists)
test_df['Count'] = test_df['Count'].apply(sum)
test_df['stats'] = test_df['stats'].apply(join_lists)
test_df['geometry'] = test_df['geometry_str'].apply(wkt.loads)
location_without_aid = df_merged[~df_merged['geometry'].isin(test_df['geometry'])][['Aid', 'Vehicle', 'Count', 'geometry']]
location_without_aid['geometry_str'] = location_without_aid['geometry'].astype(str)
location_without_aid['stats'] = 'No aid'
location_without_aid = location_without_aid[['geometry_str', 'Aid', 'Vehicle', 'Count', 'stats', 'geometry']]
location_without_aid
result_df = pd.concat([test_df,location_without_aid])
result_df

,geometry_str,Aid,Vehicle,Count,stats,geometry
0,POINT (-8.496595 31.112307),الرعاية الصحية,شاحنة صغيرة,1,الرعاية الصحية : شاحنة صغيرة (1),POINT (-8.496595 31.112307)


# Reduce duplicates

In [247]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from scipy.spatial.distance import cdist
from haversine import haversine

# Sample data with a 'geometry' column containing Shapely Point objects

gdf = gpd.GeoDataFrame(df_location, geometry='geometry')

# Create a DataFrame with coordinates
coordinates_df = gdf['geometry'].apply(lambda geom: (geom.x, geom.y)).apply(pd.Series)
coordinates_df.columns = ['longitude', 'latitude']

# Calculate the pairwise distances between points using Haversine
distances = pd.DataFrame(columns=coordinates_df.index, index=coordinates_df.index)

for i in coordinates_df.index:
    for j in coordinates_df.index:
        distances.loc[i, j] = haversine((coordinates_df.at[i, 'latitude'], coordinates_df.at[i, 'longitude']),
                                        (coordinates_df.at[j, 'latitude'], coordinates_df.at[j, 'longitude']))

# Create a mask to filter out points that are less than 5 km apart
mask = (distances >= 3) | (distances == 0)

# Apply the mask to the DataFrame
filtered_gdf = gdf[mask.any(axis=1)].reset_index(drop=True)

filtered_gdf


,name,latitude,longitude,geometry
0,Douar Tazloumt,30.808782,-8.606285,POINT (-8.60628 30.80878)
1,Talat N'Yaaqoub,30.993130,-8.235383,POINT (-8.23538 30.99313)
2,Ijoukak,30.997324,-8.162536,POINT (-8.16254 30.99732)
3,دوار امندار‎,30.847017,-8.579649,POINT (-8.57965 30.84702)
4,دوار تجكَالت‎,31.060757,-8.855264,POINT (-8.85526 31.06076)
...,...,...,...,...
328,مجموعة مدارس تكنزا - وحدة أنومس‎,30.875672,-8.787370,POINT (-8.78737 30.87567)
329,مسجد انادن,30.983637,-8.152778,POINT (-8.15278 30.98364)
330,Agdim,31.306409,-6.854532,POINT (-6.85453 31.30641)
331,Tazelga,31.237979,-7.879984,POINT (-7.87998 31.23798)


In [225]:
df_location

,name,latitude,longitude,geometry
0,Douar Tazloumt,30.808782,-8.606285,POINT (-8.606285 30.808782)
1,Talat N'Yaaqoub,30.993130,-8.235383,POINT (-8.235383 30.99313)
2,Ijoukak,30.997324,-8.162536,POINT (-8.162536 30.997324)
3,دوار امندار‎,30.847017,-8.579649,POINT (-8.579649 30.847017)
4,دوار تجكَالت‎,31.060757,-8.855264,POINT (-8.855264 31.060757)
...,...,...,...,...
328,مجموعة مدارس تكنزا - وحدة أنومس‎,30.875672,-8.787370,POINT (-8.78737 30.875672)
329,مسجد انادن,30.983637,-8.152778,POINT (-8.152778 30.983637)
330,Agdim,31.306409,-6.854532,POINT (-6.854532 31.306409)
331,Tazelga,31.237979,-7.879984,POINT (-7.879984 31.237979)


In [254]:
from sources.utils import merge_by_distance_coords, transform_df
df_merged = merge_by_distance_coords(df,df_location,3)
df_merged = transform_df(df_merged)
df_merged

KeyError: 'Longitude'